Nhập data

In [1]:
import os
import fasttext
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy
from joblib import dump, load

In [8]:
data = open('topic_detection_test.v1.0.txt', newline=None, encoding='utf-8')
# data = open('__label__tai_chinh.txt', newline=None, encoding='utf-8')
data.read()
data.seek(0)
lines = data.readlines()
lowercaseLines = list(map(lambda line: line.lower(), lines))

In [9]:
linesWithoutLinks = list(map(lambda line: re.sub(r"https?\s*:\s*\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", '', line), lowercaseLines))

In [10]:
linesFormattedDot = list(map(lambda line: re.sub(r"\s*\.+\s*(\.*\s*){100}", ' punc ', line), linesWithoutLinks))

In [11]:
linesWithoutWierdChar = list(map(lambda line: re.sub(r'[\d\W]+', ' ', line), linesFormattedDot))

In [24]:
# formattedLines = list(map(lambda line: line.replace('\n', '').split(" ", 1), lines))
formattedLines = list(map(lambda line: line.replace('\n', '').split(" ", 1), linesWithoutWierdChar))

In [27]:
import pandas as pd

In [36]:
# df = pd.DataFrame(formattedLines, columns=['label', 'content'])
df = pd.DataFrame(linesWithoutWierdChar, columns=['content'])

In [38]:
datalist = df['content'].tolist();
linesWithoutUnderScores = list((map(lambda line: re.sub(r'_+', ' ', line), datalist)))

In [1370]:
def convert_brief_char_to_word(matchobj):
    string = matchobj.group(0)
    foundNum = re.findall('\d+', string);
    result = string
    if len(foundNum) > 0:
        num = foundNum[0]
        left = " " + foundNum[1] + " " if len(foundNum) > 1 else ""
        result = num
        if len(num) < len(string):
            char = string[len(num):len(string)] if left == "" else string[len(num):len(string) - len(left) + 2]
            obj = {
                'đ': 'đồng',
                'k': 'nghìn',
                'tr': 'triệu',
                'h': 'giờ',
                'm': 'mét',
                'm²': 'mét vuông',
                'pn': 'phòng ngủ',
            }
            if obj.get(char) != None:
                result += " " + obj.get(char) + left
            else:
                result += " " + char + left
    return result

In [1371]:
splittedNumAndWordLines = list((map(lambda line: re.sub(r'\d*[a-zđ²]+\d*', convert_brief_char_to_word,line), linesWithoutUnderScores)))

In [1372]:
def convert_num_to_word(matchobj):
    num = matchobj.group(0)
    if len(num) > 7 or num[0] == '0':
        num = " ".join(list(map(lambda char: num2words(int(char), lang='vi'),list(num))))
        return num;
    return num2words(int(num), lang='vi')

In [1373]:
num2wordsLines = list((map(lambda line: re.sub(r'\d+', convert_num_to_word,line), splittedNumAndWordLines)))

In [39]:
# removedOneWordLines = list((map(lambda line: re.sub(r'\s+\w\s+', " ",line), num2wordsLines)))
removedOneWordLines = list((map(lambda line: re.sub(r'\s+\w\s+', " ",line), linesWithoutUnderScores)))

Skip 

In [232]:
def get_stopwords(documents, threshold=3):
    """
    :param documents: list of documents
    :param threshold:
    :return: list of words has idf <= threshold
    """
    tfidf = TfidfVectorizer(min_df=100)
    tfidf_matrix = tfidf.fit_transform(documents)
    features = tfidf.get_feature_names()
    stopwords = []
    print(min(tfidf.idf_), max(tfidf.idf_), len(features))
    for index, feature in enumerate(features):
        if tfidf.idf_[index] <= threshold:
            stopwords.append(feature)
    return stopwords

In [233]:
stopwords = get_stopwords(df['content'], threshold=2.5)

1.8107271245353596 6.065285982427615 3205


In [234]:
with open('manual_stopwords.txt', 'w', encoding='utf8') as fp:
        for word in stopwords:
            fp.write(word + '\n')

Continue here

In [40]:
lines = open('stopwords.txt', newline=None, encoding='utf-8')
lines.read()
lines.seek(0)
stopwords = lines.readlines()
stopwords = list(map(lambda line: line.replace('\n', ''), stopwords))

In [41]:
stopwords.pop(0)
# stopwords

'\ufeff'

In [42]:
linesWithoutStopwords = list(map(lambda line: " ".join(list(filter(lambda word: stopwords.count(word) == 0, line.split(" ")))), removedOneWordLines))

In [43]:
df['content'] = linesWithoutStopwords

Split data to files

In [1498]:
files = {}
for label, content in zip(list(df['label']), list(df['content'])):
  if files.get(label) == None:
    files[label] = []
  files[label].append(content)

In [1499]:
for label in files.keys():
  store_file(label + '.txt', files[label], len(files[label]) * [label])

In [1500]:
X_train, X_test, y_train, y_test = [[], [], [], []]
for label in files.keys():
    if "__label__tai_chinh, __label__kinh_doanh_va_cong_nghiep".count(label):
        x_tr, x_te, y_tr, y_te = train_test_split(files[label], len(files[label]) * [label], test_size=0.4, random_state=0)        
    else:
        x_tr, x_te, y_tr, y_te = train_test_split(files[label], len(files[label]) * [label], test_size=0.4, random_state=0)
    X_train.extend(x_tr)
    X_test.extend(x_te)
    y_train.extend(y_tr)
    y_test.extend(y_te)

In [1501]:
store_file('created_train.txt', X_train, y_train)
store_file('created_test.txt', X_test, y_test)

Use sklearn k-fold cross validation technique

In [7]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import f1_score

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.4, random_state=0)

In [13]:
store_file('created_train.txt', X_train.tolist(), y_train.tolist())
store_file('created_test.txt', X_test.tolist(), y_test.tolist())
# store_file('created_test.txt', X_test.tolist(), [""] * len(y_test.tolist()))
# store_file('created_test_label.txt', y_test.tolist(), [""] * len(y_test.tolist()))

In [16]:
pd.DataFrame(X_test, columns=['label', 'content'])

,label,content
841,NaN,"Mừng sinh nhật 5 tuổi, Sol Art tổ chức các hoạ..."
8650,NaN,“BỐ EM LÀ NGƯỜI VĨ ĐẠI NHẤT!” . Bài văn tả mẹ ...
3701,NaN,Xin phép ad cho mình đăng tuyển dụng nhé. Công...
13270,NaN,Còn có cái phòng T1 gần 40m2 khép kín khu wc r...
12349,NaN,"Chợ Nhà Đất Hà Nội ( Mua, Bán, Thuê, Cho Thuê ..."
...,...,...
11278,NaN,Kim chi dưa chuột chua cay hấp dẫn không ngờ ...
8413,NaN,[CAFÉ CEO] MỘT KHI ĐÃ BƯỚC CHÂN VÀO CÔNG TY: T...
4170,NaN,"Chính chủ cho Thuê phòng tại Phố Trung văn, nh..."
9952,NaN,Dear các bạn. Dạo gần đây An Phát nhận được 1...


Estimator for fasttext

In [224]:
class FasttextEstimator(BaseEstimator):

    def __init__(self, model_dir):
        self.model_dir = model_dir
        self.model = None

    def fit(self, features, labels):
        """
        Train fasttext on the given features and labels.
        :param features: a list of documents.
        :param labels: the list of labels associated with the list of features.
        """
        store_file(os.path.join(self.model_dir, "train.txt"), features, labels)
        fasttext.train_supervised(input=os.path.join(self.model_dir, "train.txt"),
                                  epoch=6, lr=1.0, wordNgrams=2, verbose=1, minCount=1, loss='hs', thread=4).save_model('cv_model.bin')
        self.model = fasttext.load_model(os.path.join(self.model_dir, "cv_model.bin"))
        return self

    def score(self, features, labels):
        """
        Compute the macro-f1 score for the predictions on the given features.
        :param features: a list of documents.
        :param labels: the list of labels associated with the list of features.
        :return: f1_score: the macro-f1 score for the predictions on the given features.
        """
        predicted_labels = []
        for feature in features:
            prediction = self.model.predict(feature)
            predicted_label = prediction[0]
            predicted_labels.append(predicted_label)
        return f1_score(labels, predicted_labels, average="macro")

In [55]:
def store_file(output_file, features, labels):
    """Write the training data in fasttext format to disk.
    :param output_file: the name of the output file.
    :param features: the features, a list of strings.
    :param labels: the labels associated with features.
    """
    with open(output_file, 'w', newline='\n', encoding='utf-8') as f:
        for i in range(0, len(features)):
            if labels[i] == '':
                f.write("%s\n" % (features[i]))
            else:
                f.write("%s %s\n" % (labels[i], features[i]))                

Fasttext library

In [55]:
# model = fasttext.train_supervised(input='created_train.txt', autotuneValidationFile='created_test.valid')
# model.save_model('cv_model.bin')

In [67]:
model = fasttext.load_model("fasttext.bin")
# model.test('created_test.txt')

In [68]:
model.labels

['__label__Nha_dat',
 '__label__Kinh_doanh_va_Cong_nghiep',
 '__label__Do_an_va_do_uong',
 '__label__Tai_chinh',
 '__label__Mua_sam',
 '__label__Du_lich',
 '__label__Chinh_tri',
 '__label__Giao_duc',
 '__label__Nghe_thuat',
 '__label__Mang_internet_va_vien_thong',
 '__label__Sach',
 '__label__Con_nguoi_va_xa_hoi',
 '__label__Lam_dep_va_the_hinh',
 '__label__Nha_va_vuon',
 '__label__Giai_tri',
 '__label__May_tinh_va_thiet_bi_dien_tu',
 '__label__Suc_khoe_va_benh_tat',
 '__label__Thoi_quen_va_so_thich',
 '__label__Khoa_hoc',
 '__label__Phap_luat',
 '__label__Giao_thong',
 '__label__The_thao',
 '__label__Cong_nghe_moi']

In [52]:
predictions = []
for content in df['content'].tolist():
    predictions.append(model.predict(content)[0][0])

In [56]:
store_file('predictions_fasttext.txt', predictions, [""] * len(predictions))

In [26]:
# model.('mùa thứ nhất')
estimator = FasttextEstimator(model_dir='C:/Users/Dung/Desktop/python/My python note/')
cross_val_score(estimator, df['content'].tolist(), df['label'].tolist(), cv=5)





C:\Users\Dung\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)

C:\Users\Dung\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([0.83949904, 0.8426808 , 0.83133947, 0.82731172, 0.82239846])

In [1]:
# model = fasttext.load_model("cv_model.bin")
# model.get_words()

Transform word to vectors

Use a vectorizer

In [20]:
Tfidf_vect = TfidfVectorizer(max_features=5000, ngram_range=(1,2), sublinear_tf=True)
Tfidf_vect.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [21]:
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [180]:
# print(Train_X_Tfidf)

SVM

In [25]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [183]:
# param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf']}
# grid = GridSearchCV(svm.SVC(),param_grid,refit = True, verbose=2, cv=4)
# grid.fit(Train_X_Tfidf,y_train)
# grid.best_params_

Use svm after gridsearchCV

In [381]:
# predictions_SVM = grid.predict(Test_X_Tfidf)

Use svm manually

In [26]:
SVM = svm.SVC(C=1, gamma=1, kernel='linear')
SVM.fit(Train_X_Tfidf, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [27]:
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [28]:
print(classification_report(y_test,predictions_SVM))

                                       precision    recall  f1-score   support

                   __label__chinh_tri       0.97      0.98      0.97       341
         __label__con_nguoi_va_xa_hoi       0.95      0.99      0.97       161
               __label__cong_nghe_moi       1.00      0.62      0.76        13
            __label__do_an_va_do_uong       0.99      0.99      0.99       948
                     __label__du_lich       0.99      0.97      0.98       366
                    __label__giai_tri       0.97      1.00      0.99        74
                    __label__giao_duc       0.95      0.94      0.94       269
                  __label__giao_thong       1.00      1.00      1.00        37
                    __label__khoa_hoc       0.98      0.89      0.94        65
   __label__kinh_doanh_va_cong_nghiep       0.61      0.73      0.66       950
         __label__lam_dep_va_the_hinh       0.99      0.92      0.95       125
 __label__mang_internet_va_vien_thong       0.99   

In [85]:
matrix = confusion_matrix(y_test, predictions_SVM)

In [173]:
# list(enumerate(maxtrix[20].tolist())) # tai chinh bị đoán nhầm thành kinh doanh và công nghiệp

In [174]:
# list(enumerate(maxtrix[9].tolist()))

In [175]:
# list(enumerate(maxtrix[2].tolist()))

Trial

In [29]:
outlier_y_train = []
outlier_x_train = []
outlier_y_test = []
outlier_x_test = []

outliers = ", ".join(['__label__tai_chinh', '__label__kinh_doanh_va_cong_nghiep'])

for label, content in zip(y_train, X_train):
    if outliers.count(label) == 1:
        outlier_y_train.append(label)
        outlier_x_train.append(content)
for label, content in zip(y_test, X_test):
    if outliers.count(label) == 1:
        outlier_y_test.append(label)
        outlier_x_test.append(content)
temp_y_train = ['__label__outlier' if outliers.count(label) else label
               for label in y_train]
temp_y_test = ['__label__outlier' if outliers.count(label) else label
               for label in y_test]
store_file('outlier_train.txt', outlier_x_train, outlier_y_train)
store_file('outlier_test.txt', outlier_x_test, outlier_y_test)
store_file('train_temp.txt', X_train.tolist(), temp_y_train)
store_file('test_temp.txt', X_test.tolist(), temp_y_test)

In [30]:
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [31]:
SVM = svm.SVC(C=1, gamma=1, kernel='linear')
SVM.fit(Train_X_Tfidf, temp_y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [32]:
predictions_SVM = SVM.predict(Test_X_Tfidf)
print(classification_report(temp_y_test,predictions_SVM))

                                       precision    recall  f1-score   support

                   __label__chinh_tri       0.97      0.98      0.97       341
         __label__con_nguoi_va_xa_hoi       0.95      0.99      0.97       161
               __label__cong_nghe_moi       1.00      0.62      0.76        13
            __label__do_an_va_do_uong       0.99      0.99      0.99       948
                     __label__du_lich       0.99      0.97      0.98       366
                    __label__giai_tri       0.97      1.00      0.99        74
                    __label__giao_duc       0.95      0.94      0.95       269
                  __label__giao_thong       1.00      0.97      0.99        37
                    __label__khoa_hoc       0.98      0.88      0.93        65
         __label__lam_dep_va_the_hinh       0.99      0.92      0.95       125
 __label__mang_internet_va_vien_thong       1.00      0.98      0.99       233
__label__may_tinh_va_thiet_bi_dien_tu       1.00   

Outlier

In [146]:
Tfidf_vect_outlier = TfidfVectorizer(max_features=1000, ngram_range=(1,2), max_df=0.5, sublinear_tf=True)
Tfidf_vect_outlier.fit(outlier_x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=1000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [147]:
outlier_true_x_test = []
outlier_true_y_test = []
x_list = X_test.tolist()
y_list = y_test.tolist()
# for content in X_test.tolist():
#     if outlier_x_test.count(content) == 1:
#         index = outlier_x_test.index(content)
#         outlier_true_x_test.append(outlier_x_test[index])
#         outlier_true_y_test.append(outlier_y_test[index])
for index, label in enumerate(predictions_SVM):
    if label == '__label__outlier':
        outlier_true_x_test.append(x_list[index])
        outlier_true_y_test.append(y_list[index])   
store_file('outlier_true_test.txt', outlier_true_x_test, outlier_true_y_test)

In [148]:
Train_X_outlier_Tfidf = Tfidf_vect_outlier.transform(outlier_x_train)
Test_X_outlier_Tfidf = Tfidf_vect_outlier.transform(outlier_true_x_test)

In [149]:
# param_grid = {'C':range(1, 100, 5),'gamma':numpy.arange(0.5, 10, 0.5)}
# grid = GridSearchCV(svm.SVC(kernel='sigmoid'),param_grid,refit = True, verbose=2, cv=2)
# grid.fit(Train_X_Tfidf,y_train)
# grid.best_params_

In [150]:
# SVM_outlier = svm.SVC(C=1, gamma=1.8, kernel='sigmoid') # 0.4 test 
# class_weight = {'__label__cong_nghe_moi': 0.4, '__label__tai_chinh': 0.5, '__label__kinh_doanh_va_cong_nghiep':0.3} # 0.2 test
# class_weight = {'__label__tai_chinh': 0.5, '__label__kinh_doanh_va_cong_nghiep':0.35} # 0.2 test
# SVM_outlier = svm.SVC(C=4, gamma=2, kernel='sigmoid', class_weight=class_weight) 0.2 test
SVM_outlier = svm.SVC(C=1, gamma=1.8, kernel='sigmoid')
SVM_outlier.fit(Train_X_outlier_Tfidf, outlier_y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1.8, kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [151]:
predictions_SVM_outlier = SVM_outlier.predict(Test_X_outlier_Tfidf)
print(classification_report(outlier_true_y_test,predictions_SVM_outlier))

                                       precision    recall  f1-score   support

                   __label__chinh_tri       0.00      0.00      0.00         4
         __label__con_nguoi_va_xa_hoi       0.00      0.00      0.00         1
               __label__cong_nghe_moi       0.00      0.00      0.00         5
                     __label__du_lich       0.00      0.00      0.00         4
                    __label__giao_duc       0.00      0.00      0.00         6
                  __label__giao_thong       0.00      0.00      0.00         1
                    __label__khoa_hoc       0.00      0.00      0.00         4
   __label__kinh_doanh_va_cong_nghiep       0.65      0.79      0.71       916
         __label__lam_dep_va_the_hinh       0.00      0.00      0.00         3
 __label__mang_internet_va_vien_thong       0.00      0.00      0.00         3
__label__may_tinh_va_thiet_bi_dien_tu       0.00      0.00      0.00         5
                     __label__mua_sam       0.00   

C:\Users\Dung\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [197]:
# sorted(Tfidf_vect_outlier.vocabulary_, reverse=True)
Tfidf_vect_outlier.vocabulary_

{'ngắn': 453,
 'kiến': 316,
 'khai': 286,
 'vận': 863,
 'hội': 276,
 'một': 419,
 'chủ': 84,
 'cửa': 147,
 'điện': 915,
 'máy': 399,
 'đăng': 933,
 'quảng': 551,
 'cáo': 108,
 'truyền': 726,
 'hình': 245,
 'lệ': 375,
 'dân': 161,
 'động': 979,
 'viên': 843,
 'tây': 779,
 'ban': 3,
 'kỳ': 337,
 'đại': 942,
 'huy': 235,
 'chương': 71,
 'vàng': 853,
 'bất': 30,
 'mua': 396,
 'hiệu': 222,
 'tiền': 695,
 'tiêu': 684,
 'thụ': 670,
 'đoàn': 918,
 'bạc': 23,
 'toán': 706,
 'chi': 45,
 'đông': 932,
 'nhiên': 455,
 'chẳng': 78,
 'lỗ': 377,
 'chắn': 77,
 'phá': 494,
 'sản': 586,
 'lo': 355,
 'thừa': 675,
 'xu': 878,
 'tất': 800,
 'bảo': 27,
 'hiểm': 219,
 'rủi': 562,
 'ro': 557,
 'ký': 331,
 'hợp': 277,
 'đồng': 972,
 'chuyên': 55,
 'gia': 181,
 'phân': 503,
 'tích': 783,
 'thể': 660,
 'sai': 565,
 'tình': 780,
 'vui': 851,
 'triệu': 720,
 'tầm': 801,
 'cửa hàng': 148,
 'quảng cáo': 552,
 'truyền hình': 727,
 'mua hàng': 397,
 'bảo hiểm': 28,
 'rủi ro': 563,
 'hợp đồng': 280,
 'chuyên gia': 56,
 

In [153]:
predictions_SVM_outlier = predictions_SVM_outlier.tolist()
predictions = []
for label in predictions_SVM:
    if label == '__label__outlier':
        predictions.append(predictions_SVM_outlier.pop(0))
    else:
        predictions.append(label)

In [154]:
print(classification_report(y_test,predictions))

                                       precision    recall  f1-score   support

                   __label__chinh_tri       0.97      0.98      0.97       341
         __label__con_nguoi_va_xa_hoi       0.95      0.99      0.97       161
               __label__cong_nghe_moi       1.00      0.62      0.76        13
            __label__do_an_va_do_uong       0.99      0.99      0.99       948
                     __label__du_lich       0.99      0.97      0.98       366
                    __label__giai_tri       0.97      1.00      0.99        74
                    __label__giao_duc       0.95      0.94      0.95       269
                  __label__giao_thong       1.00      0.97      0.99        37
                    __label__khoa_hoc       0.98      0.88      0.93        65
   __label__kinh_doanh_va_cong_nghiep       0.65      0.76      0.70       950
         __label__lam_dep_va_the_hinh       0.99      0.92      0.95       125
 __label__mang_internet_va_vien_thong       1.00   

In [217]:
dump(SVM, 'main_model.joblib')
dump(SVM_outlier, 'side_model.joblib')

['side_model.joblib']

In [218]:
SVM = load('main_model.joblib')
SVM_outlier = load('side_model.joblib')

In [219]:
SVM

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Fasttext - test model

In [1608]:
fasttext.train_supervised(input='outlier_train.txt', maxn=0, minn=0, bucket=785249, neg=5, ws=5, dim=53,
                                  epoch=2, lr=1.0, wordNgrams=5, verbose=2, minCount=1, loss='softmax', thread=4).save_model('cv_model_outlier.bin')

In [547]:
# model = fasttext.load_model("cv_model_outlier.bin")
model.test_label('outlier_true_test.txt')

{'__label__kinh_doanh_va_cong_nghiep': {'precision': 0.5803757828810021,
  'recall': 0.5978494623655914,
  'f1score': 0.5889830508474576},
 '__label__tai_chinh': {'precision': 0.277992277992278,
  'recall': 0.26373626373626374,
  'f1score': 0.2706766917293233}}